#### Import Packages

In [1]:
from IPython.display import display, clear_output
from tkinter import Tk, filedialog
from tqdm import tqdm
import pandas as pd
import numpy as np
import csv
import re

#### Import CSV file

In [2]:
# Create a Tkinter root window
root = Tk()
root.withdraw()  # Hide the root window

# Ask the user to select a CSV file
file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])

# Check if a file was selected
if file_path:
    # Load the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path, encoding='latin-1')
    
    # Display the DataFrame
    print("CSV file loaded successfully. The shape of the data is " + str(df.shape))
    
else:
    print("No file selected.")

CSV file loaded successfully. The shape of the data is (20310, 41)


C:\Users\balaj\AppData\Local\Temp\ipykernel_2212\3719102924.py:11: DtypeWarning: Columns (19,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='latin-1')


#### Duplicate the CSV for future use

In [3]:
df_dupe = df

In [4]:
df = df_dupe

#### Drops Rows where important values are missing

In [5]:
df = df.dropna(subset=['latest_version_git_repo'])

In [6]:
df = df.dropna(subset=['Latest Version'])

In [7]:
df.shape

(10628, 41)

#### Clean Github Links

Check if 'latest_version_git_repo' is a github link. If not, drop it. Also modifies @github.com: to 'https://github.com/'

In [8]:
def clean_github_links(df):
    link_change_count = 0
    row_drop_count = 0
    for i in tqdm(df.index):
        try:
            if (re.match(r'.*(github.com).*', df['latest_version_git_repo'][i])):
                if(re.match(r'.*(@github.com:).*', df['latest_version_git_repo'][i])):
                    df.at[i, 'latest_version_git_repo'] = re.sub(r'.*(@github.com:)', "https://github.com/", df['latest_version_git_repo'][i])
                    link_change_count += 1
            else:
                df = df.drop(index=i)
                row_drop_count += 1
        except:
            df = df.drop(index=i)
            row_drop_count += 1
    clear_output(wait=True)
    print("Github links modified: " + str(link_change_count))
    print("Rows dropped: ", str(row_drop_count))
    print("New data shape: ", str(df.shape))
    return df

In [9]:
df = clean_github_links(df)

Github links modified: 103
Rows dropped:  250
New data shape:  (10378, 41)


#### Clean Question Marks

Replacing '?' with __nan__ which is displayed as blank cell in csv

In [10]:
def remove_qmarks(val):
    if(val == '?'):
        return np.NaN
    else:
        return val

In [11]:
def clean_qmarks(column_name):
    df[column_name] = df[column_name].apply(remove_qmarks)
    return df[column_name]

In [12]:
for column in df.columns:
    df[column] = clean_qmarks(column)

#### Handle 'K' in Counts

In [13]:
def modify_K(val):
    if(re.match(r'([0-9.])+(K|k)', str(val))):
        val = str(round(float(val[:-1]) * 1000))
    return val

In [14]:
def clean_k_counts(column_name):
    df[column_name] = df[column_name].apply(modify_K)
    return df[column_name]

In [15]:
columns_to_handle = ['GitHub Stars', 'Forks', 'Versions', '# of Files']
for column in columns_to_handle:
    df[column] = clean_k_counts(column)

#### Clean timestamps

In [34]:
def change_timestamps(column_name):
    df[column_name] = pd.to_datetime(df[column_name])
    df[column_name] = df[column_name].dt.strftime('%Y-%m-%d')
    return df[column_name]

In [35]:
columns_to_handle = ['modified', 'created']
for column in columns_to_handle:
    df[column] = change_timestamps(column)

#### Clean "years ago" values

In [18]:
def extract_years(value):
    try:
        match = re.search(r'(\d+)\s*(years?|months?|days?|hours?)\s*(ago)*', value)
        if match:
            num_value = int(match.group(1)) 
            unit = match.group(2)
            if unit.startswith('year'):
                return round(num_value * 12, 2)
            elif unit.startswith('month'):
                return round(num_value, 2)
            elif unit.startswith('day'):
                return round(num_value / 30.44, 2)
            elif unit.startswith('hour'):
                return round(num_value / 720, 2)
        else:
            return np.NaN
    except:
        return value

In [19]:
def handle_years(column_name):
    df[column_name] = df[column_name].apply(extract_years)
    return df[column_name]

In [20]:
columns_to_handle = ['Latest Version', 'Last Commit', 'Age', 'Last Release']
for column in columns_to_handle:
    df[column] = handle_years(column)

#### Drop 'last release'. Redundant with 'Latest Version'

In [21]:
df = df.drop('Last Release', axis = 1)
df.shape

(10378, 40)

#### Drop 'Maintainers'. Redundant with 'maintainers_length'

In [22]:
count = 0
for i in tqdm(df.index):
    if not (df['maintainers_length'][i] == str(round(df['Maintainers'][i]))):
        count += 1
clear_output(wait=True)
count

5

In [23]:
df = df.drop('Maintainers', axis = 1)
df.shape

(10378, 39)

#### Clean '-' in 'Contributors' 

In [36]:
def remove_minus(val):
    if(val == '-'):
        return np.NaN
    else:
        return val

In [37]:
def clean_minus(column_name):
    df[column_name] = df[column_name].apply(remove_minus)
    return df[column_name]

In [38]:
columns_to_handle = ['Contributors']
for column in columns_to_handle:
    df[column] = clean_minus(column)

#### Check Value Freq for a Column

In [24]:
column_name = "Maintainence"
print(df[column_name].value_counts(dropna = False))

Inactive       9926
Sustainable     312
Healthy         138
Pending          2
Name: Maintainence, dtype: int64


#### Save Cleaned Data to CSV

In [39]:
df.to_csv('cleaned_data.csv', index=False)

In [40]:
df.shape

(10378, 39)